In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector
from mysql.connector import Error

class PetAdoptionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Pet Adoption System")
        self.create_login_page()

    def create_connection(self):
        try:
            connection = mysql.connector.connect(
                host='localhost',
                database='pet_adoption',
                user='root',
                password='Mysql@123',
                auth_plugin='mysql_native_password'
            )
            if connection.is_connected():
                print("Successfully connected to the database")
                return connection
        except Error as e:
            print(f"Error while connecting to MySQL: {e}")
            return None

    def create_login_page(self):
        self.clear_frame()

        self.root.geometry("400x300")
        tk.Label(self.root, text="Login as:", font=("Arial", 16)).grid(row=0, column=0, columnspan=2, pady=20)

        tk.Button(self.root, text="Adopter", command=self.create_adopter_login_page, font=("Arial", 14), width=20, height=2).grid(row=1, column=0, padx=20, pady=20)
        tk.Button(self.root, text="Staff", command=self.create_staff_login_page, font=("Arial", 14), width=20, height=2).grid(row=1, column=1, padx=20, pady=20)

    def create_adopter_login_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Username").grid(row=0, column=0)
        tk.Label(self.root, text="Password").grid(row=1, column=0)

        self.username_entry = tk.Entry(self.root)
        self.password_entry = tk.Entry(self.root, show='*')

        self.username_entry.grid(row=0, column=1)
        self.password_entry.grid(row=1, column=1)

        tk.Button(self.root, text="Login", command=self.adopter_login).grid(row=2, column=0, columnspan=2)
        tk.Button(self.root, text="Create Account", command=self.create_adopter_account_page).grid(row=3, column=0, columnspan=2)
        tk.Button(self.root, text="Back", command=self.create_login_page).grid(row=4, column=0, columnspan=2)

    def create_staff_login_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Staff Username").grid(row=0, column=0)
        tk.Label(self.root, text="Staff Password").grid(row=1, column=0)


        self.staff_username_entry = tk.Entry(self.root)
        self.staff_password_entry = tk.Entry(self.root, show='*')

        self.staff_username_entry.grid(row=0, column=1)
        self.staff_password_entry.grid(row=1, column=1)

        tk.Button(self.root, text="Login", command=self.staff_login).grid(row=2, column=0, columnspan=2)
        tk.Button(self.root, text="Back", command=self.create_login_page).grid(row=3, column=0, columnspan=2)

    def create_adopter_account_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Username").grid(row=0, column=0)
        tk.Label(self.root, text="Password").grid(row=1, column=0)
        tk.Label(self.root, text="Email").grid(row=2, column=0)
        tk.Label(self.root, text="Phone").grid(row=3, column=0)

        self.new_username_entry = tk.Entry(self.root)
        self.new_password_entry = tk.Entry(self.root, show='*')
        self.new_email_entry = tk.Entry(self.root)
        self.new_phone_entry = tk.Entry(self.root)

        self.new_username_entry.grid(row=0, column=1)
        self.new_password_entry.grid(row=1, column=1)
        self.new_email_entry.grid(row=2, column=1)
        self.new_phone_entry.grid(row=3, column=1)

        tk.Button(self.root, text="Create Account", command=self.create_adopter_account).grid(row=4, column=0, columnspan=2)
        tk.Button(self.root, text="Back", command=self.create_adopter_login_page).grid(row=5, column=0, columnspan=2)
    def create_adopter_account(self):
        username = self.new_username_entry.get()
        password = self.new_password_entry.get()
        email = self.new_email_entry.get()
        phone = self.new_phone_entry.get()

    # Validate inputs
        if not username or not password or not email or not phone:
            messagebox.showerror("Error", "All fields must be filled")
            return

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            try:
                cursor.execute("INSERT INTO adopter (username, password, email, phno) VALUES (%s, %s, %s, %s)",
                               (username, password, email, phone))
                connection.commit()
                messagebox.showinfo("Account Creation", "Account created successfully!")
                self.create_adopter_login_page()
            except Error as e:
                messagebox.showerror("Error", f"Error creating account: {e}")
            finally:
                cursor.close()
                connection.close()

    def adopter_login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            cursor.execute("SELECT * FROM adopter WHERE username=%s AND password=%s", (username, password))
            adopter = cursor.fetchone()
            cursor.close()
            connection.close()

            if adopter:
                messagebox.showinfo("Login", "Login successful!")
                self.show_pets_page()  # Redirect to the pets page for the adopter
            else:
                messagebox.showerror("Login", "Invalid username or password")

    def staff_login(self):
        username = self.staff_username_entry.get()
        password = self.staff_password_entry.get()

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            cursor.execute("SELECT * FROM staff WHERE username=%s AND password=%s", (username, password))
            staff = cursor.fetchone()
            cursor.close()
            connection.close()

            if staff:
                messagebox.showinfo("Login", "Login successful!")
                self.show_staff_page()  # Redirect to the staff page
            else:
                messagebox.showerror("Login", "Invalid username or password")

    def show_pets_page(self):
        self.clear_frame()

        tk.Button(self.root, text="Back", command=self.create_adopter_login_page).pack(pady=10)

        self.pet_tree = ttk.Treeview(self.root, columns=('ID', 'Name', 'Breed', 'Age', 'Gender', 'Rescue Date', 'Description', 'Shelter Name', 'Shelter Address'), show='headings')

        # Define headings and columns
        headings = ['ID', 'Name', 'Breed', 'Age', 'Gender', 'Rescue Date', 'Description', 'Shelter Name', 'Shelter Address']
        widths = [30, 50, 80, 30, 50, 80, 300, 200, 300]
        for heading, width in zip(headings, widths):
            self.pet_tree.heading(heading, text=heading)
            self.pet_tree.column(heading, width=width, anchor='w')
        
        self.pet_tree.pack()
        self.load_pets()
    def show_pets_page1(self):
        self.clear_frame()

        tk.Button(self.root, text="Back", command=self.show_staff_page).pack(pady=10)

        self.pet_tree = ttk.Treeview(self.root, columns=('ID', 'Name', 'Breed', 'Age', 'Gender', 'Rescue Date', 'Description', 'Shelter Name', 'Shelter Address'), show='headings')

        # Define headings and columns
        headings = ['ID', 'Name', 'Breed', 'Age', 'Gender', 'Rescue Date', 'Description', 'Shelter Name', 'Shelter Address']
        widths = [30, 50, 80, 30, 50, 80, 300, 200, 300]
        for heading, width in zip(headings, widths):
            self.pet_tree.heading(heading, text=heading)
            self.pet_tree.column(heading, width=width, anchor='w')
        
        self.pet_tree.pack()
        self.load_pets()

    def show_staff_page(self):
        self.clear_frame()

        tk.Button(self.root, text="View Pets", command=self.show_pets_page1).pack(pady=10)
        tk.Button(self.root, text="Add Pet", command=self.add_pet_page).pack(pady=10)
        tk.Button(self.root, text="Update Pet", command=self.update_pet_page).pack(pady=10)
        tk.Button(self.root, text="Delete Pet", command=self.delete_pet_page).pack(pady=10)
        tk.Button(self.root, text="Back", command=self.create_staff_login_page).pack(pady=10)

    def add_pet_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Pet Name").pack()
        self.pet_name_entry = tk.Entry(self.root, width=50)
        self.pet_name_entry.pack()

        tk.Label(self.root, text="Breed").pack()
        self.pet_breed_entry = tk.Entry(self.root, width=50)
        self.pet_breed_entry.pack()

        tk.Label(self.root, text="Age (months)").pack()
        self.pet_age_entry = tk.Entry(self.root, width=50)
        self.pet_age_entry.pack()

        tk.Label(self.root, text="Gender").pack()
        self.pet_gender_entry = tk.Entry(self.root, width=50)
        self.pet_gender_entry.pack()

        tk.Label(self.root, text="Rescue Date").pack()
        self.pet_rescue_date_entry = tk.Entry(self.root, width=50)
        self.pet_rescue_date_entry.pack()

        tk.Label(self.root, text="Description").pack()
        self.pet_description_entry = tk.Entry(self.root, width=50)
        self.pet_description_entry.pack()

        tk.Label(self.root, text="Shelter ID").pack()
        self.pet_shelter_id_entry = tk.Entry(self.root, width=50)
        self.pet_shelter_id_entry.pack()

        tk.Button(self.root, text="Add Pet", command=self.add_pet).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_staff_page).pack(pady=5)

    def update_pet_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Pet ID").pack()
        self.pet_id_entry = tk.Entry(self.root, width=50)
        self.pet_id_entry.pack()

        tk.Label(self.root, text="New Name").pack()
        self.pet_name_entry = tk.Entry(self.root, width=50)
        self.pet_name_entry.pack()

        tk.Label(self.root, text="New Breed").pack()
        self.pet_breed_entry = tk.Entry(self.root, width=50)
        self.pet_breed_entry.pack()

        tk.Label(self.root, text="New Age (months)").pack()
        self.pet_age_entry = tk.Entry(self.root, width=50)
        self.pet_age_entry.pack()

        tk.Label(self.root, text="New Gender").pack()
        self.pet_gender_entry = tk.Entry(self.root, width=50)
        self.pet_gender_entry.pack()

        tk.Label(self.root, text="New Rescue Date").pack()
        self.pet_rescue_date_entry = tk.Entry(self.root, width=50)
        self.pet_rescue_date_entry.pack()

        tk.Label(self.root, text="New Description").pack()
        self.pet_description_entry = tk.Entry(self.root, width=50)
        self.pet_description_entry.pack()

        tk.Label(self.root, text="New Shelter ID").pack()
        self.pet_shelter_id_entry = tk.Entry(self.root, width=50)
        self.pet_shelter_id_entry.pack()

        tk.Button(self.root, text="Update Pet", command=self.update_pet).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_staff_page).pack(pady=5)

    def delete_pet_page(self):
        self.clear_frame()

        tk.Label(self.root, text="Pet ID to Delete").pack()
        self.pet_id_entry = tk.Entry(self.root, width=50)
        self.pet_id_entry.pack()

        tk.Button(self.root, text="Delete Pet", command=self.delete_pet).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_staff_page).pack(pady=5)

    def add_pet(self):
        pet_name = self.pet_name_entry.get()
        pet_breed = self.pet_breed_entry.get()
        pet_age = self.pet_age_entry.get()
        pet_gender = self.pet_gender_entry.get()
        pet_rescue_date = self.pet_rescue_date_entry.get()
        pet_description = self.pet_description_entry.get()
        shelter_id = self.pet_shelter_id_entry.get()

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            try:
                cursor.execute("INSERT INTO pet (name, breed, age, gender, rescue_date, description, shelter_id) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                               (pet_name, pet_breed, pet_age, pet_gender, pet_rescue_date, pet_description, shelter_id))
                connection.commit()
                messagebox.showinfo("Add Pet", "Pet added successfully!")
                self.show_staff_page()
            except Error as e:
                messagebox.showerror("Error", f"Error adding pet: {e}")
            finally:
                cursor.close()
                connection.close()

    def update_pet(self):
        pet_id = self.pet_id_entry.get()
        pet_name = self.pet_name_entry.get()
        pet_breed = self.pet_breed_entry.get()
        pet_age = self.pet_age_entry.get()
        pet_gender = self.pet_gender_entry.get()
        pet_rescue_date = self.pet_rescue_date_entry.get()
        pet_description = self.pet_description_entry.get()
        shelter_id = self.pet_shelter_id_entry.get()

        if not pet_id:
            messagebox.showerror("Error", "Pet ID is required")
            return

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            try:
                update_fields = []
                update_values = []

                if pet_name:
                    update_fields.append("name=%s")
                    update_values.append(pet_name)
                if pet_breed:
                    update_fields.append("breed=%s")
                    update_values.append(pet_breed)
                if pet_age:
                    update_fields.append("age=%s")
                    update_values.append(pet_age)
                if pet_gender:
                    update_fields.append("gender=%s")
                    update_values.append(pet_gender)
                if pet_rescue_date:
                    update_fields.append("rescue_date=%s")
                    update_values.append(pet_rescue_date)
                if pet_description:
                    update_fields.append("description=%s")
                    update_values.append(pet_description)
                if shelter_id:
                    update_fields.append("shelter_id=%s")
                    update_values.append(shelter_id)

                if update_fields:
                    update_query = f"UPDATE pet SET {', '.join(update_fields)} WHERE id=%s"
                    update_values.append(pet_id)
                    cursor.execute(update_query, update_values)
                    connection.commit()
                    messagebox.showinfo("Update Pet", "Pet updated successfully!")
                else:
                    messagebox.showinfo("Update Pet", "No fields to update")

                self.show_staff_page()
            except Error as e:
                messagebox.showerror("Error", f"Error updating pet: {e}")
            finally:
                cursor.close()
                connection.close()


    def delete_pet(self):
        pet_id = self.pet_id_entry.get()

        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            try:
                cursor.execute("DELETE FROM pet WHERE id=%s", (pet_id,))
                connection.commit()
                messagebox.showinfo("Delete Pet", "Pet deleted successfully!")
                self.show_staff_page()
            except Error as e:
                messagebox.showerror("Error", f"Error deleting pet: {e}")
            finally:
                cursor.close()
                connection.close()

    def load_pets(self):
        connection = self.create_connection()
        if connection:
            cursor = connection.cursor()
            cursor.execute("""
                SELECT p.id, p.name, p.breed, p.age, p.gender, p.rescue_date, p.description, s.name AS shelter_name, s.address AS shelter_address
                FROM pet p
                JOIN shelter s ON p.shelter_id = s.id
            """)
            pets = cursor.fetchall()
            cursor.close()
            connection.close()

            for pet in pets:
                self.pet_tree.insert('', 'end', values=pet)

    def clear_frame(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# Running the application
if __name__ == "__main__":
    root = tk.Tk()
    app = PetAdoptionApp(root)
    root.mainloop()
